# Werchter




# 1. Setup and Imports
This section imports the required libraries for web scraping, text processing, and working with JSON.

In [24]:
import json  # Import the JSON library to work with JSON data (e.g., read/write JSON files).
import unicodedata  # Import the library for handling Unicode data (e.g., normalize text).
import re  # Import the regular expressions library for pattern matching and text processing.

# Import Selenium components for browser automation:
from selenium.webdriver.chrome.service import Service  # For managing ChromeDriver service.
from selenium.webdriver.common.by import By  # To locate elements by different strategies (e.g., ID, class, XPath).
from selenium.webdriver.support.ui import WebDriverWait  # To wait for specific conditions or elements.
from selenium.webdriver.support import expected_conditions as EC  # Define expected conditions for WebDriverWait.

# Import Chrome options and driver manager:
from selenium.webdriver.chrome.options import Options  # To configure ChromeDriver options (e.g., headless mode).
from webdriver_manager.chrome import ChromeDriverManager  # To automatically manage ChromeDriver installation.

from selenium import webdriver  # Main Selenium library to automate the Chrome browser.

from bs4 import BeautifulSoup  # Import BeautifulSoup for parsing and navigating HTML documents.

# Import exceptions for error handling in Selenium:
from selenium.common.exceptions import TimeoutException, NoSuchElementException  # Handle timeout and missing elements.


# 2. Initialize Selenium WebDriver
This code sets up the Selenium WebDriver with a headless option to avoid opening a browser window.


In [25]:
options = Options()  
# Create an instance of Chrome Options to configure browser settings.

options.add_argument("--headless")  
# Add the headless argument to run the browser in headless mode (without a visible GUI), 
# useful for running tests or scripts on servers or CI/CD pipelines.

service = Service(ChromeDriverManager().install())  
# Set up the ChromeDriver service using ChromeDriverManager, 
# which automatically downloads and sets up the appropriate version of ChromeDriver.

driver = webdriver.Chrome(service=service, options=options)  
# Initialize the Chrome WebDriver with the specified service and options.
# This launches a new instance of the Chrome browser according to the configured settings.


# 3. Extract Festival URLs
Define a function to scrape festival URLs from the main history page.

In [26]:
def extract_festival_urls(base_url):
    # Define a function to extract URLs of festival cards from the given base URL.
    
    try:
        driver.get(base_url)
        # Navigate the browser to the specified base URL using the WebDriver.

        WebDriverWait(driver, 10).until(
            # Wait for up to 10 seconds until the specified condition is met.
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card.-history"))
            # The condition is that all elements matching the CSS selector "a.card.-history" are present on the page.
        )

        # Get the page source from the browser and parse it with BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Use BeautifulSoup to select all anchor tags with the CSS class "card -history".
        cards = soup.select("a.card.-history")

        # Extract the 'href' attribute from each card, prepend the base URL, and store the URLs in a list.
        hrefs = [f"https://www.rockwerchter.be{card['href']}" for card in cards if card.get('href')]

        return hrefs  # Return the list of URLs.

    except TimeoutException:
        # Handle cases where the page load timed out or the specified elements were not found.
        print("Page load timed out or cards not found.")
        return []  # Return an empty list in case of timeout.

    except Exception as e:
        # Handle any other unexpected errors that occur during execution.
        print(f"An error occurred while extracting URLs: {str(e)}")
        return []  # Return an empty list in case of an error.


# 4. Normalize and Clean Artist Text
Define a utility function to clean and normalize the scraped artist text.

In [27]:
def clean_artist_text(text):
    """Normalize and clean the artist text."""
    
    # Normalize the text to decompose any special characters (e.g., accents) into their basic forms.
    # Replace non-breaking spaces (\u00a0) with regular spaces, and strip leading/trailing whitespace.
    text = unicodedata.normalize("NFKD", text).replace("\u00a0", " ").strip()
    
    # Remove Unicode escape sequences (e.g., \uXXXX patterns) from the text.
    text = re.sub(r'\\u[0-9a-fA-F]{4}', '', text)
    
    # Remove date patterns from the text, including day names (e.g., "Monday") and date formats.
    # Matches patterns like "Monday 12 July" or "Friday 5 May 2023".
    text = re.sub(
        r'\b(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\s+\d{1,2}\s+[A-Za-z]+(?:\s+\d{4})?\b', 
        '', 
        text
    )
    
    # Split the cleaned text by commas into individual artist names, strip extra spaces,
    # and return a list of cleaned artist names.
    return [artist.strip() for artist in text.split(", ") if artist.strip()]


# 5. Fetch Festival Data
Define a function to fetch data (year and artists) for each festival URL.

In [28]:
def fetch_festival_data(url):
    # Define a function to fetch festival data (year and artist list) from a given URL.

    try:
        # Extract the year from the URL using a regular expression.
        year_match = re.search(r'(\d{4})$', url)  
        # Search for a 4-digit number at the end of the URL.
        year = int(year_match.group(0)) if year_match else None  
        # If a match is found, convert it to an integer; otherwise, set the year to None.

        if year is None:
            # If the year could not be extracted, log a message and return None values.
            print(f"Skipping invalid URL: {url}")
            return None, None

        driver.get(url)  
        # Navigate the WebDriver to the given URL.

        WebDriverWait(driver, 2).until(
            # Wait for up to 2 seconds for the page to load.
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-component='oembed/oembed']"))
            # The condition is that an element with the specified CSS selector is present.
        )

        # Get the page source from the browser and parse it with BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find the div element with the attribute `data-component="oembed/oembed"`.
        data_div = soup.find('div', attrs={'data-component': 'oembed/oembed'})

        artist_list = []  # Initialize an empty list to store artist names.

        if data_div:
            # If the `data_div` is found, search for the first <p> tag within it.
            first_p_tag = data_div.find('p')
            if first_p_tag:
                # If the <p> tag exists, clean and extract the text using `clean_artist_text`.
                artist_list = clean_artist_text(first_p_tag.get_text(strip=True))

        # Return the year and the artist list (or None if the artist list is empty).
        return year, artist_list if artist_list else None

    except TimeoutException:
        # Handle cases where the page took too long to load.
        print(f"Failed to fetch data for {url}: Page load timed out.")
        return None, None

    except NoSuchElementException:
        # Handle cases where the required element was not found on the page.
        print(f"Failed to fetch data for {url}: Required element not found.")
        return None, None

    except Exception as e:
        # Handle any other unexpected exceptions that may occur.
        print(f"Failed to fetch data for {url}: {str(e)}")
        return None, None


# 6. Main Process to Collect Festival Data
Use the helper functions to scrape and organize festival data.

In [29]:
# Define the base URL for the Rock Werchter history page.
base_url = "https://www.rockwerchter.be/en/history"

# Call the `extract_festival_urls` function to retrieve a list of festival URLs from the base URL.
festival_urls = extract_festival_urls(base_url)

# Initialize an empty dictionary to store festival data, 
# where the key is the year and the value is the list of artists.
festival_data = {}

# Loop through each festival URL in the list of extracted URLs.
for url in festival_urls:
    # Use the `fetch_festival_data` function to extract the year and artists from the festival URL.
    year, artists = fetch_festival_data(url)
    
    # If both year and artists are successfully retrieved:
    if year and artists:
        # Add the year as a key and the list of artists as the value to the `festival_data` dictionary.
        festival_data[year] = artists


Failed to fetch data for https://www.rockwerchter.be/en/history/rock-werchter-2017: Page load timed out.


# 7. Save and Display Data
Save the collected data to a JSON file and display it.

In [30]:
# Close WebDriver
driver.quit()  
# Properly close and clean up the WebDriver instance to free resources.

# Print the scraped data in the specified format
print("\nCollected Festival Data:\n")  
# Print a header for the output.

# Loop through the festival data dictionary, sorted by year in descending order (most recent first).
for year, artists in sorted(festival_data.items(), reverse=True):
    print(f"{year}:")  
    # Print the year as the main heading.

    for artist in artists:
        # Loop through the list of artists for the given year and print each artist.
        print(f"  - {artist}")
    
    print()  
    # Add a newline after listing all artists for a particular year for better readability.

# Save the data to a JSON file
output_file = 'rockwerchter_festival.json'  
# Define the name of the output file to save the data.

try:
    with open(output_file, 'w', encoding='utf-8') as json_file:
        # Open the specified file in write mode with UTF-8 encoding.
        # Use the `json.dump` function to write the festival data dictionary to the file.
        json.dump(festival_data, json_file, ensure_ascii=False, indent=4)
        # `ensure_ascii=False` ensures that non-ASCII characters are preserved.
        # `indent=4` makes the JSON file more readable by formatting it with indentation.

    print(f"Data successfully saved to {output_file}")
    # Confirm that the data was successfully saved.

except Exception as e:
    # Handle any errors that occur during the file-writing process.
    print(f"An error occurred while saving data to JSON: {str(e)}")
    # Print the error message to help diagnose the problem.



Collected Festival Data:

2024:
  - : Lenny Kravitz
  - Greta Van Fleet
  - Dropkick Murphys
  - Parkway Drive
  - The Hives
  - The Gaslight Anthem
  - STONE
  - Jane's Addiction
  - PJ Harvey
  - Black Pumas
  - MEUTE
  - Eefje de Visser
  - Jalen Ngonda
  - The Streets
  - Nathaniel Rateliff & The Night Sweats
  - Slowdive
  - Johnny Marr
  - Bombay Bicycle Club
  - The Cat Empire
  - The Clockworks
  - Skindred
  - DEHD
  - Alice Merton
  - Kingfishr
  - PEUK

2023:
  - : Mumford & Sons
  - Stormzy
  - Sam Fender
  - The 1975
  - Froukje
  - Zwangere Guy
  - Compact Disk Dummies
  - Iggy Pop
  - Charlotte de Witte
  - Warhaus
  - King Princess
  - Anna Calvi
  - Weyes Blood
  - Röyksopp
  - Aurora
  - Raye
  - Ashnikko
  - Gayle
  - Holly Humberstone
  - Picture This
  - The Reytons
  - Lil Lotus
  - Militarie Gun
  - Body Type
  - The Mary Wallopers

2022:
  - : Pearl Jam
  - The War on Drugs
  - Pixies
  - Rag'n'Bone Man
  - HAIM
  - Fontaines D.C.
  - Gang of Youths
  - The Ki